In [2]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
# from huggingface_hub import snapshot_download
from dotenv import load_dotenv
load_dotenv()
import os


TOKEN = os.getenv("HF_TOKEN")

In [3]:
policy = SmolVLAPolicy.from_pretrained('franka_smolvla')

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Loading  HuggingFaceTB/SmolVLM2-500M-Video-Instruct weights ...
Reducing the number of VLM layers to 16 ...
Loading weights from local directory


In [4]:
# from pprint import pprint
# from gymnasium import envs
# pprint(envs.registry)

In [10]:
import gymnasium as gym
import torch
import panda_gym
import numpy as np
import imageio
from pathlib import Path

# Define output directory for video
output_directory = Path("output_videos")
output_directory.mkdir(parents=True, exist_ok=True)

# Make environment
env = gym.make("PandaPickAndPlace-v3", render_mode="rgb_array")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Print environment spaces for sanity check
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

# (Assuming `policy` is already loaded, e.g. via SmolVLAPolicy.from_pretrained(...) )
print("Policy input features:", policy.config.input_features)
print("Policy output features:", policy.config.output_features)

# Reset
policy.reset()
obs, info = env.reset(seed=42)

rewards = []
frames = [env.render()]
done = False
step = 0

while not done:
    # Debug observation structure
    print("Observation keys:", obs.keys())

    # Convert observation to tensors
    obs_raw = obs["observation"]
    state = torch.tensor(obs_raw[0:13], dtype=torch.float32).to(device).unsqueeze(0)
    goal = torch.tensor(obs["desired_goal"], dtype=torch.float32).to(device).unsqueeze(0)
    img = torch.tensor(frames[-1], dtype=torch.float32).permute(2,0,1).unsqueeze(0).to(device)/255

    # Input format expected by policy
    observation = {
        "observation.state": state,
        "observation.images.image": img,
        "task": ["pick and place"]  # can be repeated per batch

    }

    # Get action
    with torch.inference_mode():
        action = policy.select_action(observation)

    # Step
    numpy_action = action.squeeze(0).cpu().numpy()
    actions = [numpy_action[i:i+4] for i in range(0, len(numpy_action), 4)]
    print(actions)
    for action in actions:
        if action.shape[0] == 4:
            obs, reward, terminated, truncated, info = env.step(action)

        rewards.append(reward)
        frames.append(env.render())

        print(f"Step {step} | Reward: {reward:.3f} | Terminated: {terminated}")
        done = terminated or truncated
        step += 1

# Final result
print("🎯 Success!" if terminated else "❌ Failure.")
print(f"Total reward: {sum(rewards):.2f}")

# Save video
fps = env.metadata.get("render_fps", 30)
video_path = output_directory / "rollout.mp4"
imageio.mimsave(str(video_path), np.stack(frames), fps=fps)
print(f"🎥 Video saved to: {video_path}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Observation space: Dict('achieved_goal': Box(-10.0, 10.0, (3,), float32), 'desired_goal': Box(-10.0, 10.0, (3,), float32), 'observation': Box(-10.0, 10.0, (19,), float32))
Action space: Box(-1.0, 1.0, (4,), float32)
Policy input features: {'observation.images.image_additional_view': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 128, 128)), 'observation.images.image': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 128, 128)), 'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(13,))}
Policy output features: {'action': PolicyFeature(type=<FeatureType.ACTION: 'ACTION'>, shape=(15,))}
Observation keys: dict_keys(['observation', 'achieved_goal', 'desired_goal'])
[array([-0.01062162,  0.00829344,  0.00525099,  0.02169818], dtype=float32), array([ 0.01869066,  0.05501999, -0.03474769,  0.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🎥 Video saved to: output_videos/rollout.mp4


In [ ]:
observation = {
        "observation.state": torch.from_numpy(np.random.rand(13)).unsqueeze(0).to(device,dtype=torch.float32),
        "observation.images.image": torch.from_numpy(np.random.rand(3, 128, 128)).unsqueeze(0).to(device,dtype=torch.float32),
        "task": ["pick and place"]  # can be repeated per batch

    }

# Get action
with torch.inference_mode():
    action = policy.select_action(observation)


In [13]:
action

tensor([[ 0.0199, -0.0012, -0.0131, -0.0218,  0.0343,  0.0290,  0.0217, -0.0078,
          0.0016, -0.0061,  0.0027, -0.0411,  0.0020, -0.9401, -0.0018]])